## Data exploration

In [1]:
import pandas as pd
# import plotly 
import os
import pyarrow as pa
import numpy as np
import pandas as pd
import csv

try:
    print(run_only_once)
except Exception as e:
    print(os.getcwd())
    os.chdir("./../..")
    print(os.getcwd())
    run_only_once = "Dir has already been changed"

C:\Users\Zan\Desktop\NTNU\TDT4310_Intelligent_text_analysis_language_understanding\ITAaLU_project\notebook\data_exploration
C:\Users\Zan\Desktop\NTNU\TDT4310_Intelligent_text_analysis_language_understanding\ITAaLU_project


In [2]:
def prepare_hatespeech_v2_dataset(save=True):
    hate_df = pd.read_csv("data/toraman22_hate_speech_v2/Toraman22_hate_speech_v2.tsv", sep="\t")
    dataset_v2 = pd.read_csv("data\hatespeech_v2\hate_speech_dataset_v2.csv")
    
    print("Starting size of full dataset", hate_df.shape)
    hate_df = hate_df.drop_duplicates()
    print("Removed duplicates size of dataset", hate_df.shape)
    
    indexes_to_drop = []
    for index, row in hate_df.iterrows():
        try:
            favourite_int = int(row["tweet_id"])
        except:
            indexes_to_drop.append(index)
    
    print(f"Found {len(indexes_to_drop)} rows to drop")
    hate_df = hate_df.drop(indexes_to_drop)
    hate_df["tweet_id"] = hate_df['tweet_id'].astype(str)
    new_size = hate_df.shape
    print(f"New size is {new_size}")

    # verify that the IDs are present in the dataset on Github
    dataset_v2["TweetID"] = dataset_v2['TweetID'].astype(str)
    out_df = hate_df[hate_df["tweet_id"].isin(dataset_v2["TweetID"])]
    assert new_size == out_df.shape, "Problems occured while filtering! Sizes should match!"
    assert 128907 == out_df.shape[0], f"Size doesn't match with the required one! It should be 128907, but is {out_df.shape[0]}!"
    
    # filter by english
    out_df = out_df[out_df["language"] == 1].reset_index(drop=True)
    assert out_df.shape[0] == 68597, "Size of the final processed dataset isn't correct, errors occured!"
    print(f"Final dataset size is {out_df.shape}")

    # remove large spaces
    out_df["text"] = out_df["text"].str.replace(r"\s+", " ", regex=True)

    if save:
        save_file = "data/hatespeech_v2/prepared_hatespeech_v2.csv"
        print("Saving data")
        out_df.to_csv(save_file, index=False)
        print(f"Saved preprocessed data to: {save_file}")

In [3]:
prepare_hatespeech_v2_dataset()

C:\Users\Zan\AppData\Local\Temp\ipykernel_7812\449982524.py:2: DtypeWarning: Columns (0,4,10) have mixed types. Specify dtype option on import or set low_memory=False.
  hate_df = pd.read_csv("data/toraman22_hate_speech_v2/Toraman22_hate_speech_v2.tsv", sep="\t")


Starting size of full dataset (162386, 20)
Removed duplicates size of dataset (128917, 20)
Found 10 rows to drop
New size is (128907, 20)
Final dataset size is (68597, 20)
Saving data
Saved preprocessed data to: data/hatespeech_v2/prepared_hatespeech_v2.csv
